In [1]:
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
from shapely import wkt
from cartoframes.viz import *
import networkx as nx

In [ ]:
# 데이터 로드
dong_boundary = gpd.read_file('../emd2/행정구역.shp', encoding='CP949')

# 서울시 동 정보 추출
dong1 = dong_boundary[dong_boundary['ADM_CD'].str.startswith('11')]

# CRS 변환
dong1.set_crs(epsg=5179, inplace=True)
dong1.to_crs(epsg=4326, inplace=True)

# 결과 확인
print(dong1.crs)
print(dong1.shape)

EPSG:4326
(424, 7)


In [8]:
dong_boundary = gpd.read_file('../emd/BND_ADM_DONG_PG.shp', encoding='CP949')
dong2 = dong_boundary[dong_boundary['ADM_CD'].str.startswith('11')]   # 서울시내 동 정보 추출
dong2.to_crs(epsg=4326, inplace=True)

# 중복이름 동 방지
dong2.loc[(dong2['ADM_NM']=='신사동') & (dong2['ADM_CD']=='11230510'), 'ADM_NM'] = '신사동(강남)'
dong2.loc[(dong2['ADM_NM']=='신사동') & (dong2['ADM_CD']=='11210680'), 'ADM_NM'] = '신사동(관악)'

dong2.head()

/home/hj/.local/lib/python3.10/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,BASE_DATE,ADM_NM,ADM_CD,geometry
0,20230701,사직동,11010530,"POLYGON ((126.97399 37.57823, 126.97400 37.578..."
1,20230701,삼청동,11010540,"POLYGON ((126.97929 37.58349, 126.97931 37.583..."
2,20230701,부암동,11010550,"POLYGON ((126.96173 37.60714, 126.96182 37.607..."
3,20230701,평창동,11010560,"POLYGON ((126.97508 37.63118, 126.97488 37.630..."
4,20230701,한남동,11030740,"POLYGON ((127.00457 37.54980, 127.00457 37.549..."


In [ ]:
dong1['ADM_NM']

In [6]:
# layer_dong = Layer(dong,
#                     basic_style(opacity=0.2), # 투명도
#                     popup_hover=popup_element('ADM_NM')) # 얹으면 팝업

# layer_dong

In [7]:
dong

,SIDO_CD,SIDO_NM,SIGUNGU_CD,SIGUNGU_NM,ADM_CD,ADM_NM,geometry
0,11,서울특별시,11010,종로구,1101053,사직동,"POLYGON ((126.97399 37.57823, 126.97400 37.578..."
1,11,서울특별시,11010,종로구,1101054,삼청동,"POLYGON ((126.97714 37.59768, 126.97730 37.597..."
2,11,서울특별시,11010,종로구,1101055,부암동,"POLYGON ((126.96253 37.60575, 126.96246 37.605..."
3,11,서울특별시,11010,종로구,1101056,평창동,"POLYGON ((126.97508 37.63118, 126.97468 37.629..."
4,11,서울특별시,11010,종로구,1101057,무악동,"POLYGON ((126.96079 37.57378, 126.96074 37.573..."
...,...,...,...,...,...,...,...
419,11,서울특별시,11250,강동구,1125070,둔촌1동,"POLYGON ((127.14480 37.51939, 127.14483 37.519..."
420,11,서울특별시,11250,강동구,1125071,둔촌2동,"POLYGON ((127.15671 37.53758, 127.15654 37.537..."
421,11,서울특별시,11250,강동구,1125072,암사1동,"POLYGON ((127.13522 37.55473, 127.13569 37.554..."
422,11,서울특별시,11250,강동구,1125073,천호2동,"POLYGON ((127.11633 37.55005, 127.11642 37.550..."


# 동코드 확인

In [5]:
dong_mapping = pd.read_excel('../Dataset/dong_mapping.xlsx')
dong_mapping.columns = dong_mapping.iloc[0]
dong_mapping['H_SDNG_CD'] = dong_mapping['H_SDNG_CD'].astype(str)
dong_mapping = dong_mapping.iloc[1:].reset_index(drop=True)
dong_mapping.loc[dong_mapping['H_SDNG_CD']=='1121068', 'H_DNG_NM'] = '신사동(관악)'
dong_mapping.loc[dong_mapping['H_SDNG_CD']=='1123051', 'H_DNG_NM'] = '신사동(강남)'
dong_mapping    # H_SDNG_CD: 통계청코드,	H_DNG_CD: 행안부코드

,H_SDNG_CD,H_DNG_CD,DO_NM,CT_NM,H_DNG_NM
0,1101053,11110530,서울,종로구,사직동
1,1101054,11110540,서울,종로구,삼청동
2,1101055,11110550,서울,종로구,부암동
3,1101056,11110560,서울,종로구,평창동
4,1101057,11110570,서울,종로구,무악동
...,...,...,...,...,...
419,1125070,11740690,서울,강동구,둔촌1동
420,1125071,11740700,서울,강동구,둔촌2동
421,1125072,11740570,서울,강동구,암사1동
422,1125073,11740610,서울,강동구,천호2동


In [6]:
# dong_mapping 기준으로 맞추기
map_dict = {'종로1.2.3.4가동': '종로1·2·3·4가동',
            '종로5.6가동': '종로5·6가동',
            '금호2.3가동': '금호2·3가동',
            '면목3.8동': '면목3·8동',
            '상계3.4동': '상계3·4동',
            '상계6.7동': '상계6·7동',
            '중계2.3동': '중계2·3동'
            }

# 이름 일치 시키기
for key, val in map_dict.items():
    dong_mapping.loc[dong_mapping['H_DNG_NM']==key, 'H_DNG_NM'] = val

# 일원2동 -> dong에 없음
# 상일동 -> 상일1동, 상일2동
# 항동 -> dongmapping에 없음
# 개포3동 -> dongmapping에 없음

code2name_dict = dong_mapping.set_index('H_DNG_CD')['H_DNG_NM'].to_dict()
#code_name_dict

In [7]:
dm_i = pd.merge(dong_mapping, dong, left_on='H_DNG_NM', right_on='ADM_NM', how='inner')
dm_o = pd.merge(dong_mapping, dong, left_on='H_DNG_NM', right_on='ADM_NM', how='outer')
dm_l = pd.merge(dong_mapping, dong, left_on='H_DNG_NM', right_on='ADM_NM', how='left')
dm_r = pd.merge(dong_mapping, dong, left_on='H_DNG_NM', right_on='ADM_NM', how='right')

print(f"inner: {len(dm_i)}")
print(f"outer: {len(dm_o)}")
print(f"left: {len(dm_l)}")
print(f"right: {len(dm_r)}")

inner: 422
outer: 428
left: 424
right: 426


In [8]:
not_in_left = list(dm_l.loc[dm_l['ADM_NM'].isna(), 'H_DNG_NM'])
display(dm_l.loc[dm_l['ADM_NM'].isna()])

not_in_right = list(dm_r.loc[dm_r['H_DNG_NM'].isna(), 'ADM_NM'])
display(dm_r.loc[dm_r['H_DNG_NM'].isna()])

# 일원2동 -> 
# 항동 -> 오류2동 값 반으로 나누기
# 개포3동 -> 개포2동의 값을 반으로 나누기
# 상일1, 2동 -> 상일동의 값을 반으로 나누기

,H_SDNG_CD,H_DNG_CD,DO_NM,CT_NM,H_DNG_NM,BASE_DATE,ADM_NM,ADM_CD,geometry
372,1123074,11680740,서울,강남구,일원2동,NaN,NaN,NaN,None
407,1125052,11740520,서울,강동구,상일동,NaN,NaN,NaN,None


,H_SDNG_CD,H_DNG_CD,DO_NM,CT_NM,H_DNG_NM,BASE_DATE,ADM_NM,ADM_CD,geometry
319,NaN,NaN,NaN,NaN,NaN,20230701,항동,11170740,"POLYGON ((126.82439 37.48884, 126.82409 37.488..."
380,NaN,NaN,NaN,NaN,NaN,20230701,개포3동,11230511,"POLYGON ((127.08338 37.49994, 127.08348 37.499..."
418,NaN,NaN,NaN,NaN,NaN,20230701,상일1동,11250760,"POLYGON ((127.17387 37.54591, 127.17390 37.545..."
419,NaN,NaN,NaN,NaN,NaN,20230701,상일2동,11250770,"POLYGON ((127.18162 37.55837, 127.18162 37.558..."


In [9]:
dong[dong['ADM_NM'].str.contains('개포')]

,BASE_DATE,ADM_NM,ADM_CD,geometry
1621,20230701,개포1동,11230680,"POLYGON ((127.05826 37.48474, 127.05924 37.483..."
1623,20230701,개포4동,11230710,"POLYGON ((127.05176 37.48479, 127.05199 37.484..."
1628,20230701,개포3동,11230511,"POLYGON ((127.08338 37.49994, 127.08348 37.499..."
1636,20230701,개포2동,11230800,"POLYGON ((127.07277 37.49191, 127.07290 37.491..."


In [10]:
dong[dong['ADM_NM'].str.contains('일원')]

,BASE_DATE,ADM_NM,ADM_CD,geometry
1624,20230701,일원본동,11230720,"POLYGON ((127.09012 37.49099, 127.09012 37.490..."
1625,20230701,일원1동,11230730,"POLYGON ((127.09949 37.49284, 127.09950 37.492..."


In [11]:
dong_mapping[dong_mapping['H_DNG_NM'].str.contains('상일')]

,H_SDNG_CD,H_DNG_CD,DO_NM,CT_NM,H_DNG_NM
407,1125052,11740520,서울,강동구,상일동


# Pablo Data (Human Flow)

In [73]:
# 파블로 데이터 수정
'''
1. 코드 -> 동이름 변경 (없는 코드 찾아서 입력)
2. dong 데이터프레임에 없는 이름 처리

'''
hf_df = pd.read_csv('../Dataset/PabloTask_AggregatedDataframe.csv')
hf_df['행정동코드'] = (hf_df['행정동코드'].astype(int)).astype(str)

not_match_dong_dict = {
    "11350605": "상계6·7동",
    "11110660": "이화동",
    "11140610": "신당5동",
    "11140620": "황학동",
    "11140630": "중림동",
    "11140640": "동화동",
    "11140660": "약수동",
    
}   # dong_mapping에 있는 이름으로 변경 -> 여기에 있는 값들은 모두 0인 행들이 많음 (중복되는 이름으로 존재!!)

strange_dong_names = list(not_match_dong_dict.values())
strange_dong_codes = list(not_match_dong_dict.keys())

dong_codes = list(hf_df['행정동코드'].unique())

# 동이름으로 변경
for i in range(len(dong_codes)):
    # 사전에 있는 이름일때
    if dong_codes[i] in code2name_dict:
        hf_df.loc[hf_df['행정동코드']==dong_codes[i], '행정동'] = code2name_dict[dong_codes[i]]   # 동이름으로 변경
    elif dong_codes[i] in not_match_dong_dict:
        hf_df.loc[hf_df['행정동코드']==dong_codes[i], '행정동'] = not_match_dong_dict[dong_codes[i]]   # 동이름으로 변경



In [74]:
hf_df[hf_df['행정동코드'].isin(strange_dong_codes)]

,기준일ID,행정동코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,남자30세부터34세생활인구수,남자35세부터39세생활인구수,...,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수,총생활인구수_LONG_FOREIGNER,중국인체류인구수_LONG_FOREIGNER,중국외외국인체류인구수_LONG_FOREIGNER,총생활인구수_TEMP_FOREIGNER,중국인체류인구수_TEMP_FOREIGNER,중국외외국인체류인구수_TEMP_FOREIGNER,행정동
133,201701,11350605,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,상계6·7동
376,201701,11110660,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,이화동
395,201701,11140610,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,신당5동
396,201701,11140620,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,황학동
397,201701,11140630,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,중림동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28313,202207,11140610,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,신당5동
28314,202207,11140620,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,황학동
28315,202207,11140630,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,중림동
28316,202207,11140640,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,동화동


In [75]:
hf_df[hf_df['행정동코드'].isin(strange_dong_codes) & hf_df['총생활인구수']>0]    # 이상치로 껴있는 값인것으로 확인

,기준일ID,행정동코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,남자30세부터34세생활인구수,남자35세부터39세생활인구수,...,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수,총생활인구수_LONG_FOREIGNER,중국인체류인구수_LONG_FOREIGNER,중국외외국인체류인구수_LONG_FOREIGNER,총생활인구수_TEMP_FOREIGNER,중국인체류인구수_TEMP_FOREIGNER,중국외외국인체류인구수_TEMP_FOREIGNER,행정동


In [ ]:
filtered_df = hf_df[~hf_df['행정동코드'].isin(strange_dong_codes)]   # 이상한 값을 제외한 결과

filtered_df

,기준일ID,행정동코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,남자30세부터34세생활인구수,남자35세부터39세생활인구수,...,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수,총생활인구수_LONG_FOREIGNER,중국인체류인구수_LONG_FOREIGNER,중국외외국인체류인구수_LONG_FOREIGNER,총생활인구수_TEMP_FOREIGNER,중국인체류인구수_TEMP_FOREIGNER,중국외외국인체류인구수_TEMP_FOREIGNER,행정동
0,201701,11680660,13280.254999,0.021818,0.013837,0.026797,0.034385,0.032238,0.033898,0.034266,...,0.053565,0.032346,0.070751,195.668856,0.577567,0.422433,65.363134,0.173782,0.826223,개포1동
1,201701,11680670,22332.541530,0.018449,0.015521,0.034912,0.036598,0.030069,0.028870,0.031961,...,0.048585,0.035763,0.067330,194.587204,0.507402,0.492598,60.153137,0.093206,0.906797,개포2동
2,201701,11680690,19291.974860,0.025808,0.016365,0.026257,0.033729,0.037979,0.044007,0.048349,...,0.043502,0.028968,0.054162,261.182516,0.552713,0.447287,105.935856,0.161913,0.838091,개포4동
3,201701,11680521,46264.099476,0.011242,0.004582,0.012902,0.041534,0.069857,0.073809,0.067674,...,0.027895,0.022511,0.051450,1730.469307,0.576033,0.423967,2655.690724,0.308662,0.691340,논현1동
4,201701,11680531,44025.228452,0.017398,0.006485,0.014074,0.030946,0.057355,0.063417,0.068922,...,0.031869,0.024065,0.048626,1402.016862,0.527876,0.472124,1004.179389,0.406923,0.593079,논현2동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28336,202207,11260590,26250.251355,0.018768,0.007940,0.016566,0.032964,0.049103,0.046916,0.048295,...,0.035183,0.028312,0.060851,670.164781,0.648107,0.351893,70.571753,0.233554,0.763086,상봉2동
28337,202207,11260680,34433.186458,0.055620,0.022878,0.023058,0.025568,0.028871,0.027180,0.035146,...,0.040565,0.026446,0.076454,364.838736,0.573119,0.426881,86.290023,0.277698,0.720521,신내1동
28338,202207,11260690,17140.233495,0.037573,0.021887,0.029926,0.024963,0.025679,0.028551,0.037314,...,0.043381,0.030919,0.082931,145.204337,0.439366,0.560634,42.427717,0.074040,0.919352,신내2동
28339,202207,11260600,16755.466004,0.051979,0.020656,0.023558,0.025592,0.032629,0.032814,0.041342,...,0.040876,0.033689,0.084199,215.486106,0.538155,0.461845,35.123075,0.415183,0.574480,중화1동


In [77]:
# 기준일ID로 그룹화하고 각 그룹의 데이터 개수 확인
grouped_counts = filtered_df.groupby('기준일ID').size()

non_416_days = grouped_counts[grouped_counts != 416]
print(non_416_days)

Series([], dtype: int64)


In [78]:
filtered_dong_names = set(filtered_df['행정동'].unique())
dong_mapping_dongs = set(dong_mapping['H_DNG_NM'])

no_data_dongs = list(dong_mapping_dongs - filtered_dong_names)  # 데이터가 없는 동들

In [79]:
dong_mapping[dong_mapping['H_DNG_NM'].isin(no_data_dongs)]

,H_SDNG_CD,H_DNG_CD,DO_NM,CT_NM,H_DNG_NM
27,1102069,11140615,서울,중구,신당동
28,1102070,11140625,서울,중구,다산동
29,1102071,11140635,서울,중구,약수동
30,1102072,11140645,서울,중구,청구동
31,1102073,11140665,서울,중구,동화동
175,1111079,11350595,서울,노원구,공릉1동
357,1123051,11680510,서울,강남구,신사동(강남)
405,1124082,11710647,서울,송파구,위례동


In [80]:
strange_dong_codes

['11350605',
 '11110660',
 '11140610',
 '11140620',
 '11140630',
 '11140640',
 '11140660']

In [81]:
hf_df[hf_df['행정동코드']=='11140615']

,기준일ID,행정동코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,남자30세부터34세생활인구수,남자35세부터39세생활인구수,...,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수,총생활인구수_LONG_FOREIGNER,중국인체류인구수_LONG_FOREIGNER,중국외외국인체류인구수_LONG_FOREIGNER,총생활인구수_TEMP_FOREIGNER,중국인체류인구수_TEMP_FOREIGNER,중국외외국인체류인구수_TEMP_FOREIGNER,행정동


In [82]:
# Step 1. Build the "Dong Code x Dong name" conversion table
file_path = "../Pablo/Dong_conversion_codes.xlsx"
dataframe_Code2Dong = pd.read_excel(file_path, usecols=["행정동코드","행정동"])

print(dataframe_Code2Dong.shape)

#Filtering, remove duplicates
dataframe_Code2Dong.drop_duplicates(inplace = True, subset='행정동', keep="last")
dataframe_Code2Dong.dropna(inplace = True)
print(dataframe_Code2Dong.shape)

(732, 2)
(423, 2)


In [83]:
dataframe_Code2Dong

,행정동코드,행정동
0,11680660.0,개포1동
2,11680670.0,개포2동
3,11680690.0,개포4동
4,11680521.0,논현1동
5,11680531.0,논현2동
...,...,...
725,11260590.0,상봉제2동
726,11260680.0,신내1동
727,11260690.0,신내2동
728,11260600.0,중화제1동


In [84]:
ex_df

,기준일ID,시간대구분,행정동코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,남자30세부터34세생활인구수,...,여자25세부터29세생활인구수,여자30세부터34세생활인구수,여자35세부터39세생활인구수,여자40세부터44세생활인구수,여자45세부터49세생활인구수,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수
0,20170101,0,11110515,16846.9908,551.1680,351.2056,369.8000,1304.6177,850.2599,754.2146,...,476.3511,461.1296,625.6547,740.2699,756.2081,573.9838,473.3860,430.2336,350.5058,1206.3597
1,20170101,0,11110530,25606.3679,725.4355,322.4155,532.2063,1224.8244,1204.7807,1075.2772,...,1122.1443,890.0688,973.3847,980.6140,1161.5489,845.4654,749.9668,716.6214,498.2217,1844.1171
2,20170101,0,11110540,5724.9509,87.0075,66.6684,105.3054,524.3655,509.7949,358.1338,...,167.9036,155.2524,159.2981,153.4277,185.8502,157.4214,158.8650,146.5678,127.6351,447.9828
3,20170101,0,11110550,11691.6393,300.6513,233.1209,322.6515,411.5216,323.3993,295.2344,...,287.1489,331.9336,365.2763,512.1536,554.7413,456.3265,501.4379,437.0760,386.6049,1296.1468
4,20170101,0,11110560,24976.4805,590.4086,427.0619,631.2858,742.1176,616.4945,550.0281,...,598.5069,651.1402,897.7209,1010.2025,1112.7978,1069.6627,1170.9235,1090.1355,743.0981,3009.9850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315451,20170131,23,11740650,30818.0154,978.2184,339.3317,677.4612,1047.8307,1387.0332,1340.7342,...,1433.8173,1302.3924,1396.1695,991.2961,1104.1640,1263.5940,1506.2611,1404.9240,940.1157,1952.6181
315452,20170131,23,11740660,32966.8778,878.9140,528.3139,995.3519,1170.6795,1325.6534,1445.5374,...,1369.4592,1420.2780,1540.8826,1211.4598,1318.6189,1336.8815,1636.3228,1392.7521,973.7139,1761.6372
315453,20170131,23,11740685,61867.8410,2151.6114,1040.4466,1449.9768,1720.4191,2209.2321,2579.1172,...,2332.1165,2460.6187,2641.6134,2337.8502,2629.2590,2420.6461,3254.9028,2574.5404,1978.5046,3650.4622
315454,20170131,23,11740690,16507.3545,391.9036,297.0229,689.9862,660.0864,588.6716,415.1893,...,519.3581,399.6859,465.8694,479.5920,910.4639,919.9470,1077.2810,784.1080,493.7634,968.2621


In [85]:
file_path = "../Pablo/datasets/LOCAL_PEOPLE_DONG/LOCAL_PEOPLE_DONG_201701.csv"
ex_df = pd.read_csv(file_path)

ex_df['행정동코드'] = ex_df['행정동코드'].astype(str)

dongs_ex = list(ex_df['행정동코드'].unique())
print(len(dongs_ex))


424


In [86]:
d_names = []
for d in dongs_ex:
    if d in code2name_dict:
        d_names.append(code2name_dict[d]) 
    else:
        print(d)

print(len(d_names))

424


In [87]:
code2name_dict

{'11110530': '사직동',
 '11110540': '삼청동',
 '11110550': '부암동',
 '11110560': '평창동',
 '11110570': '무악동',
 '11110580': '교남동',
 '11110600': '가회동',
 '11110615': '종로1·2·3·4가동',
 '11110630': '종로5·6가동',
 '11110640': '이화동',
 '11110670': '창신1동',
 '11110680': '창신2동',
 '11110690': '창신3동',
 '11110700': '숭인1동',
 '11110710': '숭인2동',
 '11110515': '청운효자동',
 '11110650': '혜화동',
 '11140520': '소공동',
 '11140540': '회현동',
 '11140550': '명동',
 '11140570': '필동',
 '11140580': '장충동',
 '11140590': '광희동',
 '11140605': '을지로동',
 '11140650': '신당5동',
 '11140670': '황학동',
 '11140680': '중림동',
 '11140615': '신당동',
 '11140625': '다산동',
 '11140635': '약수동',
 '11140645': '청구동',
 '11140665': '동화동',
 '11170510': '후암동',
 '11170520': '용산2가동',
 '11170530': '남영동',
 '11170570': '원효로2동',
 '11170580': '효창동',
 '11170590': '용문동',
 '11170630': '이촌1동',
 '11170640': '이촌2동',
 '11170650': '이태원1동',
 '11170660': '이태원2동',
 '11170690': '서빙고동',
 '11170700': '보광동',
 '11170555': '청파동',
 '11170560': '원효로1동',
 '11170625': '한강로동',
 '11170685': '한남동',
 '11200